## 1 - PreProcessing


### 1.a) obtaing the raw test and train data from the text file 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np


import re


raw_train_data = []  # the raw train data we elicit from the text file
with open('/content/drive/My Drive/NSL-KDD/KDDTrain+.txt', 'r') as f:
  for line in f.readlines():
    raw_train_data.append(line.split(","))
    raw_train_data[-1][-1] = raw_train_data[-1][-1].rstrip() # eliminating the \n from the last element
    for i,element in enumerate(raw_train_data[-1]):
      # converting the int and floats from string
      if element.isdigit(): # int
        raw_train_data[-1][i] = int(element)
      elif re.match(r'^-?\d+(?:\.\d+)$', element): #float
        raw_train_data[-1][i] = float(element)
header_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack_type', 'success_pred']
train_df = pd.DataFrame(data = raw_train_data , columns=header_names )


######## test data

raw_test_data = []  # the raw test data we elicit from the text file
with open('/content/drive/My Drive/NSL-KDD/KDDTest+.txt', 'r') as f:
  for line in f.readlines():
    raw_test_data.append(line.split(","))
    raw_test_data[-1][-1] = raw_test_data[-1][-1].rstrip() # eliminating the \n from the last element
    for i,element in enumerate(raw_test_data[-1]):
      # converting the int and floats from string
      if element.isdigit(): # int
        raw_test_data[-1][i] = int(element)
      elif re.match(r'^-?\d+(?:\.\d+)$', element): #float
        raw_test_data[-1][i] = float(element)
header_names = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells', 'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate', 'attack_type', 'success_pred']
test_df = pd.DataFrame(data = raw_test_data , columns=header_names )

train_df.head(10)


    
  

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,num_failed_logins,logged_in,num_compromised,root_shell,su_attempted,num_root,num_file_creations,num_shells,num_access_files,num_outbound_cmds,is_host_login,is_guest_login,count,srv_count,serror_rate,srv_serror_rate,rerror_rate,srv_rerror_rate,same_srv_rate,diff_srv_rate,srv_diff_host_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack_type,success_pred
0,0,tcp,ftp_data,SF,491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,0.0,0.0,0.0,0.0,1.00,0.00,0.00,150,25,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,normal,20
1,0,udp,other,SF,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,1,0.0,0.0,0.0,0.0,0.08,0.15,0.00,255,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal,15
2,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,123,6,1.0,1.0,0.0,0.0,0.05,0.07,0.00,255,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,19
3,0,tcp,http,SF,232,8153,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,5,5,0.2,0.2,0.0,0.0,1.00,0.00,0.00,30,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal,21
4,0,tcp,http,SF,199,420,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,30,32,0.0,0.0,0.0,0.0,1.00,0.00,0.09,255,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal,21
5,0,tcp,private,REJ,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,121,19,0.0,0.0,1.0,1.0,0.16,0.06,0.00,255,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,neptune,21
6,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,166,9,1.0,1.0,0.0,0.0,0.05,0.06,0.00,255,9,0.04,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
7,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,117,16,1.0,1.0,0.0,0.0,0.14,0.06,0.00,255,15,0.06,0.07,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
8,0,tcp,remote_job,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,270,23,1.0,1.0,0.0,0.0,0.09,0.05,0.00,255,23,0.09,0.05,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21
9,0,tcp,private,S0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,8,1.0,1.0,0.0,0.0,0.06,0.06,0.00,255,13,0.05,0.06,0.00,0.00,1.00,1.00,0.00,0.00,neptune,21


### 1.b) converting categorical features to one hot encodings



####1.b.1) extracting categorical columns from data

In [ ]:
# converting the categorical features like protocal_type to numbers

# Categorical boolean mask
categorical_feature_mask = train_df.dtypes==object
# filter categorical columns using mask and turn it into a list
categorical_columns = train_df.columns[categorical_feature_mask].tolist()
print(categorical_columns)

# inserting the categicoral columns in a different table
train_df_categorical_values = train_df[categorical_columns]
test_df_categorical_values = test_df[categorical_columns]
train_df_categorical_values.head(10)



['protocol_type', 'service', 'flag', 'attack_type']


,protocol_type,service,flag,attack_type
0,tcp,ftp_data,SF,normal
1,udp,other,SF,normal
2,tcp,private,S0,neptune
3,tcp,http,SF,normal
4,tcp,http,SF,normal
5,tcp,private,REJ,neptune
6,tcp,private,S0,neptune
7,tcp,private,S0,neptune
8,tcp,remote_job,S0,neptune
9,tcp,private,S0,neptune


####1.b.2) creating label for each of the categories

In [ ]:
# creating column label for each of the possible categories
# protocol type
unique_protocol=sorted(train_df.protocol_type.unique())
string1 = 'Protocol_type_'
unique_protocol2=[string1 + x for x in unique_protocol]
print(unique_protocol2)

# service
unique_service=sorted(train_df.service.unique())
string2 = 'service_'
unique_service2=[string2 + x for x in unique_service]
print(unique_service2)


# flag
unique_flag=sorted(train_df.flag.unique())
string3 = 'flag_'
unique_flag2=[string3 + x for x in unique_flag]
print(unique_flag2)


#attack type

unique_attack=sorted(train_df.attack_type.unique())
string1 = 'attack_type_'
unique_attack2=[string1 + x for x in unique_attack]
print(unique_attack2)

# the columns we will add to our new table
dumcols = {'protocol_type':unique_protocol2, 'service':unique_service2, 'flag':unique_flag2, 'attack_type':unique_attack2}



#do it for test set
# in our test set, some rows have attack types not present in training data. thus we will remove them
unique_attack_test=sorted(test_df.attack_type.unique())
attack_differences = []
for st in unique_attack_test:
  if st not in unique_attack:
    attack_differences.append(st)
print(attack_differences)
print(test_df.shape)
for difference in attack_differences:
  test_df = test_df[test_df.attack_type != difference]
print(test_df.shape)
# also some rows have nans. we will delete them to
test_df = test_df.dropna()
# reseting the indexes
test_df = test_df.reset_index(drop=True)
print(test_df.shape)
# after omitting the redundant rows reseting the categorical values for tests
test_df_categorical_values = test_df[categorical_columns] 
testdumcols= dumcols

['Protocol_type_icmp', 'Protocol_type_tcp', 'Protocol_type_udp']
['service_IRC', 'service_X11', 'service_Z39_50', 'service_aol', 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns', 'service_ctf', 'service_daytime', 'service_discard', 'service_domain', 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i', 'service_efs', 'service_exec', 'service_finger', 'service_ftp', 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames', 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001', 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell', 'service_ldap', 'service_link', 'service_login', 'service_mtp', 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn', 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u', 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3', 'service_printer', 'service_private', 'service_red_i', 'service_remote_job', 'ser

#### 1.b.3)Transform categorical features into numbers using LabelEncoder()


In [ ]:
from sklearn import preprocessing


print("before:")

print(train_df_categorical_values.head())
print('--------------------')
print(test_df_categorical_values.head())
print('--------------------')
for label in categorical_columns:
  print(label)
  le = preprocessing.LabelEncoder()
  le.fit(train_df_categorical_values[label]) # fitting the incoder with the column in training data
  # digitizing the specified column in test data
  train_df_categorical_values[label] = le.transform(train_df_categorical_values[label])
  # digitizing the specified column in test data
  test_df_categorical_values[label] = le.transform(test_df_categorical_values[label])

print("after:")

print(train_df_categorical_values.head())
print('--------------------')
print(test_df_categorical_values.head())
print('--------------------')








before:
  protocol_type   service flag attack_type
0           tcp  ftp_data   SF      normal
1           udp     other   SF      normal
2           tcp   private   S0     neptune
3           tcp      http   SF      normal
4           tcp      http   SF      normal
--------------------
  protocol_type   service flag attack_type
0           tcp   private  REJ     neptune
1           tcp   private  REJ     neptune
2           tcp  ftp_data   SF      normal
3           tcp      http   SF      normal
4           tcp      smtp   SF      normal
--------------------
protocol_type
service
flag
attack_type
after:
   protocol_type  service  flag  attack_type
0              1       20     9           11
1              2       44     9           11
2              1       49     5            9
3              1       24     9           11
4              1       24     9           11
--------------------
   protocol_type  service  flag  attack_type
0              1       49     1            9
1      

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


####1.b.4) one hot encoding the data


In [ ]:
from sklearn.preprocessing import OneHotEncoder

print("before:")
print("shapes:")
print(train_df.shape)
print(test_df.shape)
print(train_df.head())
print('--------------------')
print(test_df.head())
print('--------------------')
for label in categorical_columns:
  print(label)
  encoder = OneHotEncoder()
  encoder.fit(train_df_categorical_values[label].to_numpy().reshape(-1, 1))
  #encoding the train data
  transformed_train = encoder.transform(train_df_categorical_values[label].to_numpy().astype('int32').reshape(-1, 1))
  ohe_df = pd.DataFrame(transformed_train.toarray(),columns = dumcols[label])
  #changing ohe_df to int
  float_col = ohe_df.select_dtypes(include=['float64']) # This will select float columns only
  for col in float_col.columns.values:
    ohe_df[col] = ohe_df[col].astype('int32')
  train_df = train_df.join(ohe_df)
  train_df.drop(label, axis=1, inplace=True)
  #encoding the test data
  # print(sorted(train_df_categorical_values.flag.unique()))
  transformed_test = encoder.transform(test_df_categorical_values[label].to_numpy().astype('int32').reshape(-1, 1))
  test_ohe_df = pd.DataFrame(transformed_test.toarray(),columns = dumcols[label])

  # print(test_df.shape)
  # print(test_ohe_df.shape)
  # print("******************")
  float_col_test = test_ohe_df.select_dtypes(include=['float64']) # This will select float columns only
  for col_test in float_col_test.columns.values:
    test_ohe_df[col_test] = test_ohe_df[col_test].astype('int32')
  test_df = test_df.join(test_ohe_df)
  test_df.drop(label, axis=1, inplace=True)
# is_NaN = test_df.isnull()
# row_has_NaN = is_NaN.any(axis=1)
# rows_with_NaN = test_df[row_has_NaN]
# print(rows_with_NaN)
#test_df seems to save the onehot encodings as floats. we will convert them

# for label in dumcols.keys():
#   for new_label in dumcols[label]:
#     test_df[new_label] = test_df[new_label].astype('int32')


print("after:")
print("shapes:")
print(train_df.shape)
print(test_df.shape)
print(train_df.head())
print('--------------------')
print(test_df.head())
print('--------------------')


before:
shapes:
(125973, 43)
(18794, 43)
   duration protocol_type  ... attack_type success_pred
0         0           tcp  ...      normal           20
1         0           udp  ...      normal           15
2         0           tcp  ...     neptune           19
3         0           tcp  ...      normal           21
4         0           tcp  ...      normal           21

[5 rows x 43 columns]
--------------------
   duration protocol_type  ... attack_type success_pred
0         0           tcp  ...     neptune           21
1         0           tcp  ...     neptune           21
2         2           tcp  ...      normal           21
3         0           tcp  ...      normal           21
4         0           tcp  ...      normal           21

[5 rows x 43 columns]
--------------------
protocol_type
service
flag
attack_type
after:
shapes:
(125973, 146)
(18794, 146)
   duration  src_bytes  ...  attack_type_warezclient  attack_type_warezmaster
0         0        491  ...             

###1.c) converting data to np array


In [ ]:
train_data = train_df.loc[:,train_df.columns != 'success_pred'].to_numpy()
train_labels = train_df.loc[:,train_df.columns == 'success_pred'].to_numpy().astype('int32')

print(train_data[0:5])
print(train_labels[0:5])
test_data = test_df.loc[:,test_df.columns != 'success_pred'].to_numpy()
test_labels = test_df.loc[:,test_df.columns == 'success_pred'].to_numpy().astype('int32')

print(test_data[0:5])
print(test_labels[0:5])

[[0.000e+00 4.910e+02 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 2.000e+00 2.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  1.500e+02 2.500e+01 1.700e-01 3.000e-02 1.700e-01 0.000e+00 0.000e+00
  0.000e+00 5.000e-02 0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.

### 1.d) scaling the data


In [ ]:
from sklearn.preprocessing import StandardScaler


#getting a backup
train_data_noscale = train_data.copy()
test_data_noscale = test_data.copy()


scaler = StandardScaler()
scaler.fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)
print(train_data[0:2])
# print(test_data[0:5])

[[-1.10249223e-01 -7.67859947e-03 -4.91864438e-03 -1.40888118e-02
  -8.94864220e-02 -7.73598503e-03 -9.50756715e-02 -2.70228184e-02
  -8.09261819e-01 -1.16636426e-02 -3.66518691e-02 -2.44365073e-02
  -1.23851504e-02 -2.61800242e-02 -1.86098963e-02 -4.12211976e-02
   0.00000000e+00 -2.81749392e-03 -9.75309440e-02 -7.17045492e-01
  -3.54342850e-01 -6.37209268e-01 -6.31929033e-01 -3.74362240e-01
  -3.74431603e-01  7.71283106e-01 -3.49683031e-01 -3.74559704e-01
  -3.24063093e-01 -8.18889977e-01 -7.82366826e-01 -2.80281676e-01
   6.99722578e-02 -2.89103400e-01 -6.39531905e-01 -6.24870800e-01
  -2.24532415e-01 -3.76387026e-01 -2.65429041e-01  4.76175266e-01
  -3.67554641e-01 -3.85571256e-02 -2.40795610e-02 -8.30053118e-02
  -3.98455393e-03 -8.74008009e-02 -7.52866148e-02 -7.65558238e-02
  -6.59175417e-02 -6.70020535e-02 -6.44436406e-02 -6.54910225e-02
  -6.73597456e-02 -2.78095070e-01 -5.87970494e-02 -1.94370761e-01
  -1.58232203e-01 -6.21684115e-02 -6.14566719e-02 -1.19274330e-01
  -1.18828

###1.e) define save function

In [ ]:
from joblib import dump, load

def save_sklearn(model, file_name):
  path = '/content/drive/My Drive/models/'
  dump(model, path + file_name) 


## 2 - LSTM

###2.a) defining and shaping train,validation and test data

In [ ]:
import numpy as np
#since we will modify the labels, we'll take a copy of them
# train data

X_train = np.copy(train_data[0:int(0.8 * len(train_data))])
y_train = np.copy(train_labels[0:int(0.8 * len(train_labels))])

# validation data 
X_validation = np.copy(train_data[int(0.8 * len(train_data)) : ])
y_validation = np.copy(train_labels[int(0.8 * len(train_labels)) : ])


# test data

X_test = np.copy(test_data)
y_test = np.copy(test_labels)

#shaping the data for LSTM
# labels go from [-1,21] to [0,22]
for i in range(len(y_train)):
  y_train[i] = y_train[i] + 1

for i in range(len(y_validation)):
  y_validation[i] = y_validation[i] + 1

for i in range(len(y_test)):
  y_test[i] = y_test[i] + 1
number_of_features = len(X_train[0])
X_train = X_train.reshape(-1, 1, len(X_train[0]))
X_validation = X_validation.reshape(-1, 1, len(X_validation[0]))
X_test  = X_test.reshape(-1, 1, len(X_test[0]))


###2.b) building the model


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
# Initialize LSTM model

# model.add(layers.InputLayer((146,)))
# # Add a LSTM layer with 512 internal units.
# model.add(layers.LSTM(512))

# # Add a Dense layer with 10 units.
epochs = [1000,2000]
batches = [4,16,64,128]

model = Sequential()
model.add(layers.LSTM(512, return_sequences = True,input_shape=(1,number_of_features )))
model.add(layers.Dense(10))
model.add(layers.Dense(23,activation='softmax'))
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy',metrics=['accuracy'])
model.fit(X_train, y_train, epochs= 500 ,verbose=1, batch_size=64)

LSTM_predictions = model.predict(X_test)
true_counts = 0
for i in range(len(LSTM_predictions)):
  if list(LSTM_predictions[i][0]).index(max(LSTM_predictions[i][0])) == y_test[i]:
    true_counts += 1
print("accuracy of chosen LSTM model with epochs = ",500, "and batch = ",64," : ", true_counts/len(y_test))

  


model.save('/content/drive/My Drive/models/LSTM-model')



1575/1575 [==============================] - 42s 27ms/step - loss: 0.3958 - accuracy: 0.8370
Epoch 479/500
1575/1575 [==============================] - 42s 27ms/step - loss: 0.3956 - accuracy: 0.8369
Epoch 480/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3952 - accuracy: 0.8369
Epoch 481/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3952 - accuracy: 0.8369
Epoch 482/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3948 - accuracy: 0.8370
Epoch 483/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3944 - accuracy: 0.8368
Epoch 484/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3948 - accuracy: 0.8379
Epoch 485/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3939 - accuracy: 0.8378
Epoch 486/500
1575/1575 [==============================] - 41s 26ms/step - loss: 0.3943 - accuracy: 0.8378
Epoch 487/500
1575/1575 [==========================

## 3 - SVM





###3.a) defining train,validation and test data

In [ ]:

# train data

X_train = train_data[0:int(0.8 * len(train_data))]
y_train = train_labels[0:int(0.8 * len(train_labels))].copy()

# validation data

X_validation = train_data[int(0.8 * len(train_data)) : ]
y_validation = train_labels[int(0.8 * len(train_labels)) : ].copy()

# test data

X_test = test_data
y_test = test_labels.copy()






### 3.b) defining our possible parameters

In [ ]:
param_grid = {'C': [300,400,500], 
              'kernel': ['rbf','linear'],
              'gamma' : [0.1,1, 10]
              }


### 3.c) defining the models and picking up the best based on the validation data


In [ ]:
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix 
best_model = ''
best_accuracy = 0
chosen_c = 0
chosen_kernel = ''
for kernel in param_grid['kernel']:
  for c in param_grid['C']:
    svm_model = SVC(kernel = kernel, C = c).fit(X_train, y_train) 
    #svm_predictions = svm_model.predict(X_validation) 
    empirical_score = svm_model.score(X_train, y_train) 
    validation_score = svm_model.score(X_validation, y_validation) 
    if validation_score > best_accuracy:
      best_model = svm_model
      best_accuracy = validation_score
      chosen_c = c
      chosen_kernel = kernel
    print("Accuracy of svm with " ,"kernel = ", kernel , " and C = " , str(c) ,  " on train data is : ",empirical_score)
    print("Accuracy of svm with " ,"kernel = ", kernel , " and C = " , str(c) ,  " on validation data is : ",validation_score)
    print("***************************")

print("best svm is ", "kernel = ", chosen_kernel , " and C = " , str(chosen_c))
print("accuracy of chosen svm model : ", best_model.score(X_test,y_test))
svm_predictions = best_model.predict(X_test) 
print("confusion matrix:")
print(confusion_matrix(y_test, svm_predictions) )

save_sklearn(best_model,'svm.joblib')


# ##results:
# Accuracy of svm with  kernel =  rbf  and C =  50  on train data is :  0.7858949206745217
# Accuracy of svm with  kernel =  rbf  and C =  50  on validation data is :  0.7726136138122643
# ***************************
# Accuracy of svm with  kernel =  rbf  and C =  100  on train data is :  0.7951032535353421
# Accuracy of svm with  kernel =  rbf  and C =  100  on validation data is :  0.7801944830323477
# # ***************************
# Accuracy of svm with  kernel =  rbf  and C =  200  on train data is :  0.8047651988523605
# Accuracy of svm with  kernel =  rbf  and C =  200  on validation data is :  0.7864258781504266
# ***************************

# it turns out kernel rbf with  c = 500 yields the best results

NameError: ignored

###3.d) finding best gamma

In [ ]:
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix 
gammas =  [0.01,0.1,1, 10]
best_model = ''
best_accuracy = 0
chosen_gamma = 0
for gamma in param_grid['gamma']:
  svm_model = SVC(kernel = 'rbf', C = 500,gamma = gamma).fit(X_train, y_train) 
  empirical_score = svm_model.score(X_train, y_train) 
  validation_score = svm_model.score(X_validation, y_validation) 
  if validation_score > best_accuracy:
    best_model = svm_model
    best_accuracy = validation_score
    chosen_gamma = gamma
  print("Accuracy of svm with " ,"gamma = ", gamma ,  " on train data is : ",empirical_score)
  print("Accuracy of svm with " ,"gamma = ", gamma , " on validation data is : ",validation_score)
  print("***************************")

print("best svm is ", "gamma = ", chosen_gamma)
print("accuracy of chosen svm model : ", best_model.score(X_test,y_test))
svm_predictions = best_model.predict(X_test) 
print("confusion matrix:")
print(confusion_matrix(y_test, svm_predictions) )
# save_sklearn(best_model,'svm.joblib')
# Accuracy of svm with  gamma =  1  on train data is :  0.9254261122010411
# Accuracy of svm with  gamma =  1  on validation data is :  0.8471522127406231
# Accuracy of svm with  gamma =  0.1  on train data is :  0.8615801589911659
# Accuracy of svm with  gamma =  0.1  on validation data is :  0.8233776542964873
# ***************************
# /usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
#   y = column_or_1d(y, warn=True)
# Accuracy of svm with  gamma =  0.01  on train data is :  0.8195643052358218
# Accuracy of svm with  gamma =  0.01  on validation data is :  0.7977773367731693
# ***************************

# it turns out that the best gamma is 1

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


###3.e) final model


In [ ]:
from sklearn.svm import SVC 
from sklearn.metrics import confusion_matrix 
y_train = train_labels[0:int(0.8 * len(train_labels))].reshape(int(0.8 * len(train_labels)),)
svm_model = SVC(kernel = 'rbf', C = 500, gamma = 1,verbose = True).fit(X_train, y_train)
print("accuracy of chosen svm model : ", svm_model.score(X_test,y_test))
svm_predictions = svm_model.predict(X_test) 
print("confusion matrix:")
print(confusion_matrix(y_test, svm_predictions) ) 
save_sklearn(svm_model,'svm.joblib')

[LibSVM]accuracy of chosen svm model :  0.7125146323294669
confusion matrix:
[[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    26     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     1     0    28     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0    28     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     1     1     0    88     0     0     0]
 [    0     0     0     0     1     0     0     0     0     0     0     0
      0     0     0     1     0     0    74     0     0     0]
 [    0     0     0     0     0     0     0     0     0     1     0     0
      0     0     0     0     1     0    70     0     0     0]
 [    0     0     0     0     0     0     0     0     0     0     0     0
      1     0     0     0  

##4 - Naive Bayes

### 4.a) training our model


In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
# train data


# X_train = train_data
X_train = train_data_noscale
y_train = train_labels.reshape(len(train_labels),)




# test data
X_test = test_data

y_test = test_labels

#parameters
param_grid_nb = {
    'var_smoothing': np.logspace(0,-9, num=100)
}


nbModel_grid = GridSearchCV(estimator=GaussianNB(), param_grid=param_grid_nb, verbose=1, cv=10, n_jobs=1)
nbModel_grid.fit(X_train, y_train)
print(nbModel_grid.best_estimator_)
save_sklearn(nbModel_grid,'gnb.joblib')



Fitting 10 folds for each of 100 candidates, totalling 1000 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:  8.6min finished


GaussianNB(priors=None, var_smoothing=6.579332246575682e-07)


###4.b) evaluating the model



In [ ]:
# accuracy on X_test

accuracy = nbModel_grid.score(X_test, y_test)
print("accuracy of gnb = ",accuracy)







accuracy of gnb =  0.5684260934340747


## 5 - Random Forest



### 5.a) defining train,validation and test data


In [ ]:
# train data

X_train = train_data[:int(0.8 * len(train_data))]
y_train = train_labels[:int(0.8 * len(train_labels))]

# train data

X_validation = train_data[int(0.8 * len(train_data)) :]
y_validation = train_labels[int(0.8 * len(train_labels)) : ]

# test data

X_test = test_data
y_test = test_labels


###5.b) defining possible parameters

In [ ]:



param_grid = {
 'max_depth': [10,  50, 100],
 'n_estimators': [10,50,100,200, 400, 600],}


###5.c)defining the models and picking up the best max_depth and n_estimators 



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
best_model = ''
best_accuracy = 0
chosen_max_depth = 0
chosen_n_estimators = 0
for depth in param_grid['max_depth']:
  for n in param_grid['n_estimators']:
    rf_model = RandomForestClassifier(n_estimators = n, max_depth = depth, random_state = 42).fit(X_train, y_train) 
    empirical_score = rf_model.score(X_train, y_train) 
    validation_score = rf_model.score(X_validation, y_validation) 
    if validation_score > best_accuracy:
      best_model = rf_model
      best_accuracy = validation_score
      chosen_max_depth = depth
      chosen_n_estimators = n
    print("Accuracy of rf with " ,"max depth = ", str(depth) , " and number of estimators = " , str(n) ,  " on train data is : ",empirical_score)
    print("Accuracy of svm with " ,"max depth = ", str(depth) , " and number of estimators = " , str(n),  " on validation data is : ",validation_score)
    print("***************************")

print("best rf is ", "max depth = ", chosen_max_depth , " and number of estimators = " , str(chosen_n_estimators))
print("accuracy of chosen rf model : ", best_model.score(X_test,y_test))
rf_predictions = best_model.predict(X_test) 
print("confusion matrix:")
print(confusion_matrix(y_test, rf_predictions) )



Accuracy of rf with  max depth =  100  and number of estimators =  600  on train data is :  1.0
Accuracy of svm with  max depth =  100  and number of estimators =  600  on validation data is :  0.9024012700932724
***************************
best rf is  max depth =  100  and number of estimators =  200
accuracy of chosen rf model :  0.7228370756624455
confusion matrix:
[[    0     2     3     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3     2     9     7]
 [    2     4     2     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3     1    13     4]
 [    1     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     1     0     0     3     3    20     0]
 [    2     1     0     0     1     0     0     1     0     0     0     0
      0     0     0     6     0     1    37    13    26     2]
 [    8     0     1     0     1     1     0     0     0     0     1     0
      0

###5.d) finding the best min_samples_leaf


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
best_model = ''
best_accuracy = 0
chosen_min_samples_leaf = 0
min_samples_leafs = [1, 2, 4,8]
for min_samples_leaf in min_samples_leafs:
  rf_model = RandomForestClassifier(n_estimators = 200, max_depth = 100, random_state = 42,min_samples_leaf = min_samples_leaf).fit(X_train, y_train) 
  empirical_score = rf_model.score(X_train, y_train) 
  validation_score = rf_model.score(X_validation, y_validation) 
  if validation_score > best_accuracy:
    best_model = rf_model
    best_accuracy = validation_score
    chosen_min_samples_leaf = min_samples_leaf
  print("Accuracy of rf with " ,"min samples leaf = ", min_samples_leaf  ,  " on train data is : ",empirical_score)
  print("Accuracy of svm with " ,"max depth = ", min_samples_leaf ,  " on validation data is : ",validation_score)
  print("***************************")

print("best rf is ", "min_samples_leaf = ", chosen_min_samples_leaf )
print("accuracy of chosen rf model : ", best_model.score(X_test,y_test))
rf_predictions = best_model.predict(X_test) 
print("confusion matrix:")
print(confusion_matrix(y_test, rf_predictions) )
save_sklearn(best_model,'rf.joblib')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Accuracy of rf with  min samples leaf =  1  on train data is :  1.0
Accuracy of svm with  max depth =  1  on validation data is :  0.903234768803334
***************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Accuracy of rf with  min samples leaf =  2  on train data is :  0.954464267994999
Accuracy of svm with  max depth =  2  on validation data is :  0.8858503671363366
***************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Accuracy of rf with  min samples leaf =  4  on train data is :  0.8928436761991705
Accuracy of svm with  max depth =  4  on validation data is :  0.8559237944036515
***************************


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


Accuracy of rf with  min samples leaf =  8  on train data is :  0.8465339657464923
Accuracy of svm with  max depth =  8  on validation data is :  0.8274657670172654
***************************
best rf is  min_samples_leaf =  1
accuracy of chosen rf model :  0.7228370756624455
confusion matrix:
[[    0     2     3     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3     2     9     7]
 [    2     4     2     0     0     0     0     0     0     0     0     0
      0     0     0     0     0     0     3     1    13     4]
 [    1     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     1     0     0     3     3    20     0]
 [    2     1     0     0     1     0     0     1     0     0     0     0
      0     0     0     6     0     1    37    13    26     2]
 [    8     0     1     0     1     1     0     0     0     0     1     0
      0     0     0     0     1     9    24     9    18     3]
 [   10     0     0 